## Мысль - лучшая модель lof обучается только на небольшой части выборки.
**Идея** - обучим модель на разных выборках получим коллективное предсказание и снизим variance.
- Ансамблирование - наше всё!  :)

## Нюанс.
Данные имеют временные метки. 
Соответственно, split обязательно должен быть по времени.
- и при разделении на train - test
- и при кроссвалидации.
  
Так же и нормализовывать данные необходимо после разбиения.
- однако в этом случае качество моделей падает ниже плинтуса, то на эту утечку пока забью...)

## 1. Импорты

In [1]:
# Импорт библиотек
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import clone
from sklearn.model_selection import train_test_split, StratifiedKFold, ParameterGrid
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import (precision_score, recall_score, f1_score, 
                            precision_recall_curve, auc, confusion_matrix)
import time
import warnings
warnings.filterwarnings('ignore')

# Конфигурация
RANDOM_STATE = 42
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (8, 4)

## 2. Загрузка данных

In [2]:
# поиск пути от текущего файла в папку 
# anomaly_detection_project\data\raw где лежит датасет
def find_path_within_project(project_name, path_to_data):
    current_path = os.getcwd()   # путь к текущему файлу
  
    while True:
        # Проверяем, находится ли проект в текущем каталоге
        # если найден лепим и возвращаем нужный путь к данным
        if project_name in os.listdir(current_path):
            # Строим путь до нужной папки внутри проекта
            return os.path.join(current_path, project_name, path_to_data)
        
        parent_path = os.path.dirname(current_path)
        
        if parent_path == current_path:
            raise FileNotFoundError(f"Проект '{project_name}' не найден в родительских каталогах.")
        # если проекта нет поднимаемся выше
        current_path = parent_path

# Получаем путь
full_path_to_cc_1_02_00 = find_path_within_project(
    "anomaly_detection_project", 
    os.path.join("data", "raw",'creditcard_1_02_00.csv' )
)

In [3]:
# файл creditcard_1_02_00.csv лежит в папке data/raw/
# там добавлены новые колонки времени и суммы 
# print(f"Загрузка данных из {full_path}")
df = pd.read_csv(full_path_to_cc_1_02_00)

# посмотрим ка
print("\n")
print(f"Размер данных: {df.shape[0]} строк, {df.shape[1]} столбцов")
print(f"Распределение классов:\n{df['Class'].value_counts(normalize=True)}")
print(f"\nДанные. Первые строки:")
display(df.head())
print("Список колонок без PCA: \n", *df.columns[~df.columns.str.startswith('V')])



Размер данных: 284807 строк, 35 столбцов
Распределение классов:
Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64

Данные. Первые строки:


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V25,V26,V27,V28,Amount,Class,Time_hour,Time_day,Time_abs_hour,Amount_log
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.0,0.0,0.0,5.014760
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,0.167170,0.125895,-0.008983,0.014724,2.69,0,0.0,0.0,0.0,1.305626
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,0.0,0.0,0.0,5.939276
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.0,0.0,0.0,4.824306
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.206010,0.502292,0.219422,0.215153,69.99,0,0.0,0.0,0.0,4.262539


Список колонок без PCA: 
 Time Amount Class Time_hour Time_day Time_abs_hour Amount_log


## 3. Выделение test

In [4]:
# колонка для сплита
time_column = 'Time_abs_hour'
df[time_column].min(), df[time_column].max()

(0.0, 47.0)

In [5]:
# Выделим для теста последние 8 часов
def get_train_test(df, col, train_finish,
                   train_start = 0, test_finish = 0, gap = 0,
                   easy_check =True):
    '''делим df на train и test 
        - gap -зазор между train и test (возможно и не пригодиццо)
    '''
    # определяем test_start и test_finish
    if test_finish == 0:
        test_finish = df[col].max()
    test_start = train_finish + gap + 1
    df_train = df[(df[col] >= train_start) & (df[col] <= train_finish)]
    df_test =  df[(df[col] >= test_start) & (df[col] <= test_finish)]
    if  easy_check:
        print("easy checking: \n",
              f"train min {df_train[col].min()}, train max {df_train[col].max()}\n" , 
              f"test min {df_test[col].min()}, test max {df_test[col].max()}")
    return df_train, df_test
    

# колонка для сплита
time_column = 'Time_abs_hour'
# лимит трайна
train_finish = df[time_column].max() - 8

df_train, df_test = get_train_test(df, time_column , train_finish)
print("\nОтрезали тестовую выборку.")
print(f"Размеры train:{df_train.shape} test: {df_test.shape} .")

easy checking: 
 train min 0.0, train max 39.0
 test min 40.0, test max 47.0

Отрезали тестовую выборку.
Размеры train:(224865, 35) test: (59942, 35) .


## 4. Кросс валидация + обучение

In [6]:
# домашняя кросс валидация с учётом временных меток
# возвращает позиционные индексы
class my_Time_Series_Fold_for_df:
    def __init__(self):
        self.folds = []

    def split(self, df, time_col, n_folds, rolling_window= True,  talker = True):
        '''rolling_window - тактика формирования train'''

        self.talker = talker
        # Получаем временные метки
        time_values = df[time_col]
        min_time = time_values.min()
        max_time = time_values.max()
        ############
        print(min_time, max_time)

        # Общая продолжительность временного интервала
        total_time = max_time + 1 - min_time

        # Границы фолдов
        fold_time_limits = [total_time*i/(n_folds+1) + min_time for i in range(n_folds+2)]
        if self.talker:
            print("\n*** my_Time_Series_Fold_for_df ***\n")
            print(fold_time_limits)

        for i in range(n_folds):
            # Временные границы обучающей выборки
            # в зависимости от стратегии
            train_time_start = fold_time_limits[
                    i if rolling_window else 0
                ]
            train_time_end =  fold_time_limits[i+1]

            # Временные границы тестовой выборки
            # test_time_start = train_time_end
            test_time_end = fold_time_limits[i+2]

            # Формируем обучающую и тестовую выборки
            train_idx = df[(df[time_col] >= train_time_start) & 
                            (df[time_col] < train_time_end)].index

            test_idx = df[(df[time_col] >= train_time_end) & 
                            (df[time_col] < test_time_end)].index

            # Преобразуем метки в позиции
            train_pos = df.index.get_indexer(train_idx)
            test_pos = df.index.get_indexer(test_idx)
            if self.talker:
                print(f"Номер фолда {i+1}")
                print(f"Размер train: {len(train_pos)}, test: {len(test_pos)}")

            # Добавляем пары индексов в список фолдов
            self.folds.append((train_pos, test_pos))

        return self.folds

time_separator = my_Time_Series_Fold_for_df()

In [7]:
# домашний unsupervised GridSearch для ocsvm, Isolation Forest, LOF
class my_GridSearchCV_for_df:
    '''
        Подбор гиперпараметров модели
        Разрешённые модели OCSVM, Isolation Forest, LOF(novelty = True)
        обучение возможно только на нормальных объектах или на всех
        cv - стратегия кросс валидации класс который формирует позиции 
        X_train, y_train, X_test,  y_test по временной колонке
        n_samples - определяет сколько сэмплировать объектов для обучения,
        ибо для lof и для OCSVM размер выборки критичен
        drop_columns - колонки которые не учавствуют в обучении и предсказании
        
    '''
    def __init__(self, model, param_grid, cv = time_separator,  scoring = f1_score,
                 time_col = time_column, n_folds = 3, drop_columns = list(),
                 normalizer = None, n_samples = None, talker = True,
                 unsupervised = True, random_state=None):

        self.estimator = model
        self.drop_columns = drop_columns
        self.unsupervised = unsupervised
        self.param_grid = list(ParameterGrid(param_grid))   # dict словарь параметров
        self.cv = cv                                        # стратегия кроссвалидации
        self.scoring = scoring                              # ф-я оценки (y_true, y_pred) -> float, например f1_score
        self.time_col = time_col
        self.n_folds = n_folds
        self.normalizer = normalizer                         # LOF, OCSVM хотят нормализа
        self.n_samples = n_samples
        self.talker = talker
        self.random_state = random_state
                        # стратегия кросс валидации и обучения итоговой модели

    def fit(self, X, y, rolling_window = True):
        """ Перебирает все комбинации параметров и заполняет cv_results_.
            - X - df
            - y - Series
            - rolling_window - тактика формирования train и обучения
        """

        # структуры для хранения результатов
        all_params = []
        mean_test_scores = []
        std_test_scores = []

        # получаем разбиение по фолдам
        train_test_positions = self.cv.split(X, self.time_col,
                                             self.n_folds, rolling_window, self.talker)
        
        # добавляем в удаляемые колонку по которой делили
        self.drop_columns.append(self.time_col)

        # полный перебор
        for params in self.param_grid:
            test_scores = []

            # переделано под пандас получаем позици строк , не индексы!!!
            for train_pos, test_pos in train_test_positions:
                X_train, y_train = X.drop(self.drop_columns, axis=1).iloc[train_pos] , y.iloc[train_pos]
                X_test,  y_test  = X.drop(self.drop_columns, axis=1).iloc[test_pos] ,  y.iloc[test_pos]

                # если нужно стандартизировать выборку для обучения LOF, OCSVM
                # возможно это нужно делать после усечения train но пока так...
                if self.normalizer is not None:
                    # train
                    X_sc = self.normalizer.fit_transform(X_train)
                    # обратно в df
                    X_train = pd.DataFrame(X_sc, columns=X_train.columns, index=X_train.index)
                    # test
                    X_sc = self.normalizer.transform(X_test)
                    # обратно в df для тест наверное не нужно ...
                    X_test = pd.DataFrame(X_sc, columns=X_test.columns, index=X_test.index)

                # если нужно
                if self.unsupervised:
                    # в train только нормальные объекты
                    X_train = X_train[y_train == 0]

                # если нужно уменьшить выборку для обучения LOF, OCSVM
                if self.n_samples is not None:
                    np.random.seed(self.random_state)  # фиксируем генератор
                    indices = np.random.choice(len(X_train), size=self.n_samples, replace=False)
                    X_train = X_train.iloc[indices]

                # рожаем модель
                model = clone(self.estimator).set_params(**params)
                model.fit(X_train)

                # предсказания на test с инверсией
                y_pred = np.where(model.predict(X_test) == -1, 1, 0)
                test_scores.append(self.scoring(y_test, y_pred))

            all_params.append(params)
            mean_test_scores.append(np.mean(test_scores))
            std_test_scores.append(np.std(test_scores))
            if self.talker:
                print(f"\nПараметры {params}")
                print(f"mean_scores {mean_test_scores[-1]}")
                print(f"std {std_test_scores[-1]}")

        # собираем cv_results_
        # можно докинуть потом 'rank_test_score', 
        #'split0_test_score', 'split1_test_score', ...: оценки на каждом фолде
        # 'mean_fit_time', 'mean_score_time': время обучения и оценки
        cv_results = {
            'params': all_params,
            'mean_test_score': np.array(mean_test_scores),
            'std_test_score':  np.array(std_test_scores),
        }

        # сохраняем результаты
        self.cv_results_  = cv_results

        # выбираем лучший индекс по mean_test_score
        best_idx = np.argmax(cv_results['mean_test_score'])
        self.best_params_  = all_params[best_idx]
        self.best_score_   = cv_results['mean_test_score'][best_idx]
        self.best_estimator_ = clone(self.estimator).set_params(**self.best_params_)

        # обучаем лучший на всей выборке кроме колонок которые нужно дропнуть
        X = X.drop(self.drop_columns, axis=1)
        
        # если тактика rolling_window обучаем только на данных последнего фолда
        if rolling_window:
            X, y  = X.iloc[test_pos] ,  y.iloc[test_pos]
            
        # если нужно стандартизировать выборку для обучения LOF, OCSVM
        # возможно это нужно делать после усечения train но пока так...
        if self.normalizer is not None:
            X_sc = self.normalizer.fit_transform(X)
            # обратно в df
            X = pd.DataFrame(X_sc, columns=X.columns, index=X.index)    
            
        # только нормальные)
        if self.unsupervised:
            X = X[y==0]

        # если нужно уменьшить выборку для обучения LOF, OCSVM
        if self.n_samples is not None:
            indices = np.random.choice(len(X), size=self.n_samples*2, replace=False)
            X = X.iloc[indices]
        
        self.best_estimator_.fit(X)
        return

    def predict(self, X):
        """
        Предсказывает метки для X, используя best_estimator_.
        Возвращает 1 для нормальных и -1 для аномалий, как и ocsvm
        """
        self.check_best_estimator()

        X = X.drop(self.drop_columns, axis=1)
        # если нужно стандартизироввали выборку для обучения LOF, OCSVM то
        if self.normalizer is not None:
            X = self.normalizer.transform(X) 
            
        return self.best_estimator_.predict(X)

    def decision_function(self, X):
        """
        Возвращает значение decision_function от best_estimator_.
        или 
        """
        self.check_best_estimator()

        X = X.drop(self.drop_columns, axis=1)
        # если нужно стандартизировали выборку для обучения LOF, OCSVM
        if self.normalizer is not None:
            X = self.normalizer.transform(X) 
            
        return self.best_estimator_.decision_function(X)

    def check_best_estimator(self):
        """
        проверка наличия обученого моделя
        """
        if not hasattr(self, 'best_estimator_'):
            raise AttributeError("Нужно сначала вызвать .fit()")

In [8]:
# для предсказания оставляем только колонки с PCA признаками и Amount_log ???
# Предобработка
drop_cols = ['Class']

X = df_train.drop(drop_cols, axis=1)
y = df_train['Class']

X_test = df_test.drop(drop_cols, axis=1)
y_test = df_test['Class']

In [9]:
print("Данные содержат колонки: ", X.columns)

Данные содержат колонки:  Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Time_hour', 'Time_day', 'Time_abs_hour', 'Amount_log'],
      dtype='object')


In [10]:
# для предсказания ни временные колонки ни величина транзакций нам не нужны
# кроме PCA колонок оставляем только  'Time_abs_hour' по ней будем бить на фолды
unnecessary_cols = ['Time', 'Amount', 'Time_hour', 'Time_day', 'Amount_log'] 

## пробуем ансамбль из нескольких lof

In [11]:
class Ansamble_Models:
    def __init__(self, model, n_models,
                 grid_params, time_column, 
                 n_folds, n_samples, 
                 drop_columns = unnecessary_cols,
                 grid_verbose = True,
                 normalizer = None, random_state = RANDOM_STATE):
        self.model = model
        # переход к нечётному ибо нефик
        self.n_models = n_models - n_models % 2 + 1 
        self.grid_params = grid_params
        self.grid_verbose = grid_verbose
        self.time_column = time_column
        self.n_folds = n_folds
        self.n_samples = n_samples
        self.normalizer = normalizer
        self.drop_columns = drop_columns
        self.random_state = random_state   


    def fit(self, X, y, rolling_window=False, random_shift = 2 ):
        self.models_ = []
        for i in range(self.n_models):
            grid = my_GridSearchCV_for_df(
                        model=self.model,
                        param_grid = self.grid_params, 
                        time_col = self.time_column,
                        n_folds = self.n_folds,
                        n_samples = self.n_samples,
                        normalizer = self.normalizer,
                        drop_columns = self.drop_columns,
                        random_state = self.random_state + i*random_shift,
                        talker= self.grid_verbose
                    )
            grid.fit(X, y, rolling_window=rolling_window)
            self.models_.append(grid)
            
            print('------------------------------------------')
            print( f'Лучшие параметры {i+1}_ой модели: {grid.best_params_}')
            print( f'Лучшая оценка {i+1}_ой модели: {grid.best_score_}')
            print('------------------------------------------\n')

    def check_models_(self):
        """
        проверка наличия обученого моделя
        """
        if not hasattr(self, 'models_'):
            raise AttributeError("Нужно сначала вызвать .fit()")

    def predict(self, X):
        """
        Предсказывает метки для X, используя набор моделей.
        Возвращает 1 для нормальных и -1 для аномалий,
        """
        self.check_models_()
        
        predicts = []
        for grid  in self.models_:
            predicts.append(grid.predict(X))
        
        predict = np.sign(
            np.sum(predicts, axis = 0)
        )
        return predict

    def decision_function(self, X):
        """
        Возвращает значение decision_function от models_.
        или 
        """
        self.check_models_()
        # если нужно стандартизировали выборку для обучения LOF, OCSVM
        if self.normalizer is not None:
            X = self.normalizer.transform(X) 

        decision_funcs = []
        for grid in self.models_:
            decision_funcs.append(grid.decision_function(X))
        dec_func = StandardScaler().fit_transform(np.array(decision_funcs).T).T
            
        return np.mean(dec_func, axis = 0 )

In [12]:
# Поиск гиперпараметров
lof = LocalOutlierFactor(novelty=True)

param_lof = {
    'n_neighbors': [10, 15],
    'contamination': [ 0.001]
}

grid_lof_models = Ansamble_Models(
    model = lof,
    n_models = 7,
    grid_params = param_lof,
    time_column = time_column,
    n_folds = 3,
    n_samples = 3000, 
    drop_columns = unnecessary_cols,
    normalizer = None,
    random_state = RANDOM_STATE,
    grid_verbose=False
)

grid_lof_models.fit(X, y, rolling_window=False)

0.0 39.0
------------------------------------------
Лучшие параметры 1_ой модели: {'contamination': 0.001, 'n_neighbors': 15}
Лучшая оценка 1_ой модели: 0.5135204031991282
------------------------------------------

0.0 39.0
------------------------------------------
Лучшие параметры 2_ой модели: {'contamination': 0.001, 'n_neighbors': 15}
Лучшая оценка 2_ой модели: 0.528279878450116
------------------------------------------

0.0 39.0
------------------------------------------
Лучшие параметры 3_ой модели: {'contamination': 0.001, 'n_neighbors': 15}
Лучшая оценка 3_ой модели: 0.3906518630818358
------------------------------------------

0.0 39.0
------------------------------------------
Лучшие параметры 4_ой модели: {'contamination': 0.001, 'n_neighbors': 15}
Лучшая оценка 4_ой модели: 0.37779617323014686
------------------------------------------

0.0 39.0
------------------------------------------
Лучшие параметры 5_ой модели: {'contamination': 0.001, 'n_neighbors': 10}
Лучшая оце

In [13]:
# Оценка на train
plot_pr_curve_and_conf_matrix(
    grid_lof_models, 
    X,
    y,'Кучка LocalOutlierFactor на train' 
)

NameError: name 'plot_pr_curve_and_conf_matrix' is not defined

In [ ]:
# Оценка на test
plot_pr_curve_and_conf_matrix(
    grid_lof_models, X_test,
    y_test,'Кучка LocalOutlierFactor на test' 
)

## 6. Выводы

**Датасет**
- **Объём**: 284 807 транзакций, 35 признаков.
- **Ну очень сильный дисбаланс**: 0,17% транзакций — мошеннические (Class == 1).

**Временное разбиение**: данные разделены по `Time_abs_hour`:
  - **Train**: 0–39 часов (224 865 строк).  
  - **Test**: 40–47 часов (59 942 строк).
  
**Модель**  
- ансамбль **LocalOutlierFactor**  (unsupervised).
-  обучались на разных подвыборках данных

**Метрики**  | F1   | PR AUC |
- **Train**   | 0.66 | 0.65   |
- **Test**     | 0.61 | 0.49   |
  
**Вывод**
- Ансамбль **LOF** показал ожидаемое **улучшение качества** за счёт снижения дисперсии **коллектива моделей** относительно одной модели.

**Возможные улучшения**
- При unsupervised обучении мы игнорим важные данные о целевой переменной в трейне.  Так как эти данные у нас есть нужно попробовать модели классификаци для получения лучшего результата.

## Happy end.....